In [81]:
from langchain_community.retrievers import BM25Retriever
from typing import List
import jieba
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings

In [82]:
loader = TextLoader('medical_data.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
    separators=['\n']
)
docs = text_splitter.split_documents(documents)

In [83]:
docs[0]

Document(metadata={'source': 'medical_data.txt'}, page_content="{'question': '曲匹地尔片的用法用量', 'answer': '注意：同种药品可由于不同的包装规格有不同的用法或用量。本文只供参考。如果不确定，请参看药品随带的说明书或向医生询问。口服。一次50～100mg（1-2片），3次/日，或遵医嘱。'}")

In [84]:
def preprocessing_func(text: str) -> List[str]:
    return list(jieba.cut(text))
bm25 = BM25Retriever(docs=docs,k=10)
print(bm25.k)
retriever = bm25.from_documents(docs,preprocess_func=preprocessing_func)

10


In [85]:
retriever.invoke('骨折了应该怎么办')

[Document(metadata={'source': 'medical_data.txt'}, page_content="\n{'question': '手指骨折后弯不了怎么办', 'answer': '手指骨骨折是指手部的骨折。较其他部位骨折多见，且多为多发性骨折，骨折后有典型的移位和畸形，诊断比较容易，但仍需行X线摄片，详细了解其移位情况，从而用不同的手法进行复位和固定。手指骨折后弯不了怎么办手指骨折的治疗曾因对位不佳或固定不牢固,而产生畸形愈合或者不愈合，也常因固定不当或固定时间过长而至关节囊或侧副韧带挛缩，导致关节僵硬，特别是关节附近和颈关节的骨折，常导致关节强直，严重影响手指的功能，所以手指骨折的关键在于早期的复位固定，一般错位明显的需要手术治疗，对位良好的可以保守，石膏托外固定治疗，一般恢复需要3个月左右的时间，治疗期间要逐渐进行关节功能恢复训练，预防关节僵硬。骨折后的1-2周是第一阶段，骨折部位肿胀并且有大面积淤血，经络不通，气血阻滞，此时的饮食应以活血化瘀、行气消散为主，饮食以清淡为主如蔬菜、水果、蛋类、肉类、瘦肉，忌食辛辣油腻的食物；骨折后的2-4周是第二阶段，也最为主要的阶段，症状已消退骨骼已开始恢复。饮食要保持足够的营养，满足骨骼生长需要，可以多吃富含钙质和维生素的食品，如纯牛奶、鸡蛋、豆制品、瘦肉、青菜、萝卜等，从而可以促进骨痂生长和伤口的恢复；骨折后4-8周是第三阶段，此时骨折基本愈合脉络通畅，骨骼恢复进入到最后阶段。饮食方面要以保健预防为主，可以促进牢固的骨痂生长并且使舒筋活络，使骨折部的邻近关节能够自由的活动，从而恢复往日的功能。可以吃鸡汤、羊骨汤、鱼汤等，而且饮食方面的禁忌也逐渐减少。'}"),
 Document(metadata={'source': 'medical_data.txt'}, page_content="\n{'question': '手腕骨折一直肿胀怎么办', 'answer': '手腕骨折一直肿胀被认为是血液循环不良的原因。建议首先注意休息，避免过度活动，尤其是手腕活动。白天，受影响的上肢可以挂在脖子上。每天局部敷热敷。干燥后，尝试外部消毒。同时，你可以饮用活血化瘀的药物。如果是风湿性疾病，要根据检查结果选择治疗合适的治疗方法的。手腕骨折在日常生活中较为常见，大部分受伤者是老年人，大部分患者是由于手掌落

In [86]:
from rank_bm25 import BM25Okapi
texts = [i.page_content for i in docs]
texts_processed = [preprocessing_func(t) for t in texts]
vectorizer = BM25Okapi(texts_processed)

In [87]:
vectorizer.get_top_n(preprocessing_func('骨折了应该怎么办'),texts, n=10)

["\n{'question': '手指骨折后弯不了怎么办', 'answer': '手指骨骨折是指手部的骨折。较其他部位骨折多见，且多为多发性骨折，骨折后有典型的移位和畸形，诊断比较容易，但仍需行X线摄片，详细了解其移位情况，从而用不同的手法进行复位和固定。手指骨折后弯不了怎么办手指骨折的治疗曾因对位不佳或固定不牢固,而产生畸形愈合或者不愈合，也常因固定不当或固定时间过长而至关节囊或侧副韧带挛缩，导致关节僵硬，特别是关节附近和颈关节的骨折，常导致关节强直，严重影响手指的功能，所以手指骨折的关键在于早期的复位固定，一般错位明显的需要手术治疗，对位良好的可以保守，石膏托外固定治疗，一般恢复需要3个月左右的时间，治疗期间要逐渐进行关节功能恢复训练，预防关节僵硬。骨折后的1-2周是第一阶段，骨折部位肿胀并且有大面积淤血，经络不通，气血阻滞，此时的饮食应以活血化瘀、行气消散为主，饮食以清淡为主如蔬菜、水果、蛋类、肉类、瘦肉，忌食辛辣油腻的食物；骨折后的2-4周是第二阶段，也最为主要的阶段，症状已消退骨骼已开始恢复。饮食要保持足够的营养，满足骨骼生长需要，可以多吃富含钙质和维生素的食品，如纯牛奶、鸡蛋、豆制品、瘦肉、青菜、萝卜等，从而可以促进骨痂生长和伤口的恢复；骨折后4-8周是第三阶段，此时骨折基本愈合脉络通畅，骨骼恢复进入到最后阶段。饮食方面要以保健预防为主，可以促进牢固的骨痂生长并且使舒筋活络，使骨折部的邻近关节能够自由的活动，从而恢复往日的功能。可以吃鸡汤、羊骨汤、鱼汤等，而且饮食方面的禁忌也逐渐减少。'}",
 "\n{'question': '手腕骨折一直肿胀怎么办', 'answer': '手腕骨折一直肿胀被认为是血液循环不良的原因。建议首先注意休息，避免过度活动，尤其是手腕活动。白天，受影响的上肢可以挂在脖子上。每天局部敷热敷。干燥后，尝试外部消毒。同时，你可以饮用活血化瘀的药物。如果是风湿性疾病，要根据检查结果选择治疗合适的治疗方法的。手腕骨折在日常生活中较为常见，大部分受伤者是老年人，大部分患者是由于手掌落地后摔倒造成的。大多数骨折发生在桡骨远端关节附近2厘米处，临床上称为科利尔骨折。没有及时有效的治疗，患者的腕关节会出现关节畸形、关节运动功能受限、关节疼痛等症状，给患者的日常生活带来诸多不便。因此，在诊断和治疗中必须达到正确的复位和良好的固定。骨折初期肿

In [88]:
embeddings = HuggingFaceEmbeddings(model_name='/home/user/wyf/fastchat/bge-large-zh-v1.5', model_kwargs = {'device': 'cuda:1'})

In [89]:

db = FAISS.from_documents(docs, embeddings)

In [ ]:
# db.save_local('ypur save path')

In [90]:
bm25_res = vectorizer.get_top_n(preprocessing_func('骨折了应该怎么办'),texts, n=10)
bm25_res

["\n{'question': '手指骨折后弯不了怎么办', 'answer': '手指骨骨折是指手部的骨折。较其他部位骨折多见，且多为多发性骨折，骨折后有典型的移位和畸形，诊断比较容易，但仍需行X线摄片，详细了解其移位情况，从而用不同的手法进行复位和固定。手指骨折后弯不了怎么办手指骨折的治疗曾因对位不佳或固定不牢固,而产生畸形愈合或者不愈合，也常因固定不当或固定时间过长而至关节囊或侧副韧带挛缩，导致关节僵硬，特别是关节附近和颈关节的骨折，常导致关节强直，严重影响手指的功能，所以手指骨折的关键在于早期的复位固定，一般错位明显的需要手术治疗，对位良好的可以保守，石膏托外固定治疗，一般恢复需要3个月左右的时间，治疗期间要逐渐进行关节功能恢复训练，预防关节僵硬。骨折后的1-2周是第一阶段，骨折部位肿胀并且有大面积淤血，经络不通，气血阻滞，此时的饮食应以活血化瘀、行气消散为主，饮食以清淡为主如蔬菜、水果、蛋类、肉类、瘦肉，忌食辛辣油腻的食物；骨折后的2-4周是第二阶段，也最为主要的阶段，症状已消退骨骼已开始恢复。饮食要保持足够的营养，满足骨骼生长需要，可以多吃富含钙质和维生素的食品，如纯牛奶、鸡蛋、豆制品、瘦肉、青菜、萝卜等，从而可以促进骨痂生长和伤口的恢复；骨折后4-8周是第三阶段，此时骨折基本愈合脉络通畅，骨骼恢复进入到最后阶段。饮食方面要以保健预防为主，可以促进牢固的骨痂生长并且使舒筋活络，使骨折部的邻近关节能够自由的活动，从而恢复往日的功能。可以吃鸡汤、羊骨汤、鱼汤等，而且饮食方面的禁忌也逐渐减少。'}",
 "\n{'question': '手腕骨折一直肿胀怎么办', 'answer': '手腕骨折一直肿胀被认为是血液循环不良的原因。建议首先注意休息，避免过度活动，尤其是手腕活动。白天，受影响的上肢可以挂在脖子上。每天局部敷热敷。干燥后，尝试外部消毒。同时，你可以饮用活血化瘀的药物。如果是风湿性疾病，要根据检查结果选择治疗合适的治疗方法的。手腕骨折在日常生活中较为常见，大部分受伤者是老年人，大部分患者是由于手掌落地后摔倒造成的。大多数骨折发生在桡骨远端关节附近2厘米处，临床上称为科利尔骨折。没有及时有效的治疗，患者的腕关节会出现关节畸形、关节运动功能受限、关节疼痛等症状，给患者的日常生活带来诸多不便。因此，在诊断和治疗中必须达到正确的复位和良好的固定。骨折初期肿

In [91]:
vector_res = db.similarity_search('骨折了应该怎么办', k=10)
vector_res

[Document(metadata={'source': 'medical_data.txt'}, page_content="\n{'question': '骨折术后护理', 'answer': '骨折病人术后护理要做好外固定夹板，石膏，支具的护理。要观察外固定的松紧度，骨折早期可能因整复后1到3天，肢体明显肿胀外固定太紧，要注意观察末梢血液循环的情况，及时调整绷带的松紧度一面太紧而引起肢体缺血坏死。骨折中后期因肿胀消退后，外固定可能会松动，所以必须定期复查，以免造成骨折断端移位。如果是有局部外伤引起了局部的剧烈疼痛畸形一般是有骨折了。外伤后应该首先充分休息制动，避免小腿再活动。尽快做片子检查。。如果拍片检查确定是骨折应该尽快局部外固定充分制动，避免骨折断端的活动引起出血水肿加重或引起血管神经的损伤。完全固定后消肿治疗。同时确定是否需要进一步手术固定治疗。住院环境和作息制度给病人制造一个安静、安全、舒适、卫生的住院环境，保持室内空气新鲜，帮助病人制定规律生活和作息制度，保证充足的睡眠。这也是骨折的术后护理措施。病情观察由于老年人体内器官，代偿能力差，机体易出现电解质紊乱、全身衰竭等并发症，所以老年骨折病人在手术时机体受到极大刺激后，尤其要注意生命体征及全身情况的观察，防止因骨折手术加重老年骨折病人原有的疾患。骨折的前期阶段。骨折患者这个时候骨折后刚经过治疗，骨折部位还是肿胀，有大片的淤血，经络不通，气血阻滞，这个时候的饮食应以活血化瘀，行气消散为主。饮食以清淡为主，如蔬菜、蛋类、水果、瘦肉等，忌食辛辣、燥热、油腻。这样有利于淤血的退消，脉络的通畅，比较有利于骨骼的愈合。有利于关节共能的恢复。最主要的阶段，这时患者症状基本都已经退化。骨骼开始愈合，饮食要以营养为主，以满足骨骼生长需要，促进骨骼的愈合。多吃些骨头汤，动物的肝脏，多补充一些维生素a、d、钙已及蛋白质。吃些青菜、包菜、萝卜等维生素c含量丰富的蔬菜，以促进骨痂生长和伤口愈合。'}"),
 Document(metadata={'source': 'medical_data.txt'}, page_content="{'question': '肱骨外髁颈骨折吃什么好？', 'answer': '一、肱骨外髁颈骨折食疗方(下面资料仅供参考，详细需要咨询医生)方1 赤小豆适量煎服，加赤砂糖少许温服之，本方适用于

In [92]:
def rrf(vector_results: List[str], text_results: List[str], k: int=10, m: int=60):
        """
        使用RRF算法对两组检索结果进行重排序
        
        params:
        vector_results (list): 向量召回的结果列表,每个元素是专利ID
        text_results (list): 文本召回的结果列表,每个元素是专利ID
        k(int): 排序后返回前k个
        m (int): 超参数
        
        return:
        重排序后的结果列表,每个元素是(文档ID, 融合分数)
        """
        
        doc_scores = {}
        
        # 遍历两组结果,计算每个文档的融合分数
        for rank, doc_id in enumerate(vector_results):
            doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank+m)
        for rank, doc_id in enumerate(text_results):
            doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank+m)
        
        # 将结果按融合分数排序
        sorted_results = [d for d, _ in sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)[:k]]

        return sorted_results

In [93]:
vector_results = [i.page_content for i in vector_res]
text_results = [i for i in bm25_res]
rrf_res = rrf(vector_results, text_results)
rrf_res

["\n{'question': '手指骨折后弯不了怎么办', 'answer': '手指骨骨折是指手部的骨折。较其他部位骨折多见，且多为多发性骨折，骨折后有典型的移位和畸形，诊断比较容易，但仍需行X线摄片，详细了解其移位情况，从而用不同的手法进行复位和固定。手指骨折后弯不了怎么办手指骨折的治疗曾因对位不佳或固定不牢固,而产生畸形愈合或者不愈合，也常因固定不当或固定时间过长而至关节囊或侧副韧带挛缩，导致关节僵硬，特别是关节附近和颈关节的骨折，常导致关节强直，严重影响手指的功能，所以手指骨折的关键在于早期的复位固定，一般错位明显的需要手术治疗，对位良好的可以保守，石膏托外固定治疗，一般恢复需要3个月左右的时间，治疗期间要逐渐进行关节功能恢复训练，预防关节僵硬。骨折后的1-2周是第一阶段，骨折部位肿胀并且有大面积淤血，经络不通，气血阻滞，此时的饮食应以活血化瘀、行气消散为主，饮食以清淡为主如蔬菜、水果、蛋类、肉类、瘦肉，忌食辛辣油腻的食物；骨折后的2-4周是第二阶段，也最为主要的阶段，症状已消退骨骼已开始恢复。饮食要保持足够的营养，满足骨骼生长需要，可以多吃富含钙质和维生素的食品，如纯牛奶、鸡蛋、豆制品、瘦肉、青菜、萝卜等，从而可以促进骨痂生长和伤口的恢复；骨折后4-8周是第三阶段，此时骨折基本愈合脉络通畅，骨骼恢复进入到最后阶段。饮食方面要以保健预防为主，可以促进牢固的骨痂生长并且使舒筋活络，使骨折部的邻近关节能够自由的活动，从而恢复往日的功能。可以吃鸡汤、羊骨汤、鱼汤等，而且饮食方面的禁忌也逐渐减少。'}",
 "\n{'question': '骨折术后护理', 'answer': '骨折病人术后护理要做好外固定夹板，石膏，支具的护理。要观察外固定的松紧度，骨折早期可能因整复后1到3天，肢体明显肿胀外固定太紧，要注意观察末梢血液循环的情况，及时调整绷带的松紧度一面太紧而引起肢体缺血坏死。骨折中后期因肿胀消退后，外固定可能会松动，所以必须定期复查，以免造成骨折断端移位。如果是有局部外伤引起了局部的剧烈疼痛畸形一般是有骨折了。外伤后应该首先充分休息制动，避免小腿再活动。尽快做片子检查。。如果拍片检查确定是骨折应该尽快局部外固定充分制动，避免骨折断端的活动引起出血水肿加重或引起血管神经的损伤。完全固定后消肿治疗。同时确定是否需要进一步手术固定治疗。住院环境和作息制度给病人

In [94]:
prompt = '''
任务目标：根据检索出的文档回答用户问题
任务要求：
    1、不得脱离检索出的文档回答问题
    2、若检索出的文档不包含用户问题的答案，请回答我不知道

用户问题：
{}

检索出的文档：
{}
'''

In [95]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='Qwen2-7B-Instruct', base_url='http://localhost:8000/v1', api_key='n')
res = model.invoke(prompt.format('骨折了应该怎么办', ''.join(rrf_res)))
print(res.content)

骨折后的处理和恢复涉及多方面的考虑，包括正确的治疗方法、饮食调理、休息和康复训练等。根据上述文档，我们可以总结出以下关键点：

1. **处理与固定**：
   - **手指骨折**：需要实现精确复位和牢固固定，通常采用手法复位和铝板或石膏固定。固定位置应根据骨折的位移来确定，以便恢复手指功能。
   - **跖骨骨折**：不同类型的跖骨骨折需要不同的处理方式，包括手法复位、使用固定夹板或石膏等。

2. **饮食建议**：
   - **早期**：以活血化瘀、行气的食物为主，如蔬菜、豆制品、鱼汤、鸡蛋等。
   - **中期**：摄入止痛、化瘀、促进再生的食物，如骨头汤、田七烧鸡、动物肝脏等。
   - **晚期**：滋补肝肾、益气养血、舒筋活络的食物，如老母鸡汤、猪骨汤、羊骨汤等。
   - **一般**：多吃富含纤维素的蔬菜和水果，如猕猴桃、番石榴、草莓、柿子和柑橘等，以促进消化和提供足够的维生素。

3. **康复训练**：
   - 建议在固定期间进行适量的功能锻炼，以避免关节僵硬和促进骨折愈合。
   - 注意休息，避免过度活动，尤其是对受伤部位的过度使用。

4. **并发症预防**：
   - 注意观察伤口情况，避免感染。
   - 预防便秘，保证消化系统的正常运行。
   - 防止再次受伤，特别是在康复期间。

5. **营养补充**：
   - 保证充足的蛋白质摄入，促进骨骼健康。
   - 补充维生素D、钙等微量元素，有助于骨骼愈合。

总之，骨折后的恢复需要综合治疗和细心的护理。遵循医生的指导，合理安排饮食和活动，定期复查，是确保骨折顺利恢复的关键。


In [96]:
res = model.invoke('骨折了应该怎么办')
print(res.content)

骨折是指骨头断裂或破裂的情况，需要立即得到医疗救助。以下是一些处理骨折的初步步骤：

1. **保持冷静**：首先，确保自己或患者保持冷静，减少额外的痛苦和风险。

2. **避免移动**：在没有专业医疗人员的指导下，尽量避免移动受伤的部位。因为移动可能会导致更严重的伤害。

3. **使用冰敷**：如果条件允许，可以使用冰包或冰袋覆盖在受伤部位，每次敷约15-20分钟，每隔1-2小时重复一次。冰敷有助于减轻肿胀和疼痛。

4. **固定受伤部位**：使用夹板、绷带或任何可用的材料临时固定受伤部位，以减少移动。但注意不要过紧，以免影响血液循环。

5. **寻求专业医疗帮助**：立即联系急救服务或前往最近的医院。专业医疗人员能够对骨折进行适当的诊断和治疗，包括使用X光检查确认骨折情况，并根据伤情提供必要的治疗，如复位、固定或手术。

6. **遵医嘱**：接受治疗后，按照医生的指导进行康复训练和注意事项，包括定期复查和适当的休息与活动。

请记住，上述建议不能代替专业医疗意见。在处理骨折时，最重要的步骤是尽快寻求专业的医疗帮助。
